In [1]:
import time
import re
import json
import statistics as stats
import requests as req
import numpy as np
import pandas as pd
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.common.by import By
import tqdm
import feedparser
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# Configurar opciones de Chrome
chrome_options = Options()
chrome_options.add_argument("--incognito")  # Agregamos la opción de navegación en modo incógnito
chrome_options.add_argument(r"user-data-dir=C:\\Users\\User\\AppData\\Local\\Google\\Chrome\\User Data")
chrome_options.add_argument("--remote-allow-origins=*")
chrome_options.add_experimental_option("useAutomationExtension", False) 
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_argument("user-data-dir=selenium-cookies")
# chrome_options.add_argument('--headless')  # Habilitar si no queremos ver la ventana
chrome_options.add_experimental_option("detach", True)  # Esta opción corrige el error de cierre repentino
chrome_options.add_argument('--start-minimized')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-infobars')


In [3]:
# Otras importaciones y configuraciones de Selenium
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()  # Maximizar la ventana al iniciar


In [97]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

def scrapeo_flora_data(driver, url, flora_type):
    driver.get(url)

    try:
        # Aceptar cookies si es necesario
        aceptar = driver.find_element(By.XPATH, '//*[@id="panResultados"]/div/figure')
        aceptar.click()
    except Exception as e:
        print(f'No se encontró el botón de aceptar cookies o ya se ha aceptado.')

    # Esperar un tiempo adicional después de aceptar las cookies
    time.sleep(4)

    resultados = []

    # Espera hasta que los elementos figure estén presentes y visibles
    wait = WebDriverWait(driver, 40)  # Aumentar el tiempo de espera
    wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="panResultados"]/div/figure')))
    wait.until(EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="panResultados"]/div/figure')))

    # Encuentra todos los elementos figure en el XPath especificado
    figures = driver.find_elements(By.XPATH, '//*[@id="panResultados"]/div/figure')

    for figure in figures:
        # Extrae el enlace (href) de cada figura
        href = figure.find_element(By.XPATH, './figcaption[1]/div/p[1]/a').get_attribute('href')

        # Extrae la información de título, fecha y autor de cada figura
        try:
            titulo = figure.find_element(By.XPATH, './figcaption[1]/div/p[1]/a').text.strip()
        except NoSuchElementException:
            titulo = "No se encontró información sobre el título"

        try:
            fecha = figure.find_element(By.XPATH, './figcaption[1]/div/p[2]').text.strip()
        except NoSuchElementException:
            fecha = "No se encontró información sobre la fecha"

        try:
            # Utilizar el método text en lugar de esperar a que sea visible
            autor = figure.find_element(By.XPATH, './figcaption[1]/div/p[1]').text.strip()

        except NoSuchElementException:
            autor = "No se encontró información sobre el autor"

        resultados.append([titulo, fecha, autor, flora_type, href])

    return resultados


In [90]:
def scrapeo_flora_data(driver, url, flora_type):
    driver.get(url)
    figures = driver.find_elements(By.XPATH, '//*[@id="panResultados"]/div/figure')

    resultados = []

    for figure in figures:
        titulo = figure.find_element(By.XPATH, './/figcaption[1]/div/p[1]').text.strip()
        fecha = figure.find_element(By.XPATH, './/figcaption[1]/div/p[2]').text.strip()
        autor = figure.find_element(By.XPATH, './/figcaption[1]/div/p[3]').text.strip()

        try:
            href = figure.find_element(By.XPATH, './/figcaption[1]/div/p[3]/a').get_attribute('href')
        except NoSuchElementException:
            # Manejo de excepción si no se encuentra el elemento href
            href = None

        resultados.append([titulo, fecha, autor, flora_type, href])

    return resultados


#Ejemplo de uso
url_tulipan = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_298'
url_vid_parra = 'https://www.museodelprado.es/coleccion/obras-de-arte?searchObras=vid%20/%20parra%20(vitis%20vinifera)'

flora_data_tulipan = scrapeo_flora_data(driver, url_tulipan, 'Tulipán (Tulipa Spp.)')
flora_data_vid_parra = scrapeo_flora_data(driver, url_vid_parra, 'Vid / Parra (Vitis Vinifera)')


print(flora_data_tulipan)
print(flora_data_vid_parra)


[['La Tierra', 'Óleo sobre lienzo. Siglo XVI', 'VOS, MARTIN DE', 'Tulipán (Tulipa Spp.)', 'https://www.museodelprado.es/coleccion/obras-de-arte?cidoc:p14_carried_out_by@@@pm:author@@@ecidoc:p131_E82_p102_has_title=Vos,%20Martin%20de@es'], ['Guirnalda con la Adoración de los Reyes Magos', 'Óleo sobre lámina de cobre. Siglos XVI - XVII', 'BRUEGHEL EL JOVEN, PIETER', 'Tulipán (Tulipa Spp.)', 'https://www.museodelprado.es/coleccion/obras-de-arte?cidoc:p14_carried_out_by@@@pm:author@@@ecidoc:p131_E82_p102_has_title=Brueghel%20el%20Joven,%20Pieter@es'], ['Vase of Flowers', 'Óleo sobre tabla. 1600 - 1625', 'BRUEGHEL EL VIEJO, JAN', 'Tulipán (Tulipa Spp.)', 'https://www.museodelprado.es/coleccion/obras-de-arte?cidoc:p14_carried_out_by@@@pm:author@@@ecidoc:p131_E82_p102_has_title=Brueghel%20el%20Viejo,%20Jan@es'], ['Florero', 'Óleo sobre tabla. Primer cuarto del siglo XVII', 'BRUEGHEL EL VIEJO, JAN', 'Tulipán (Tulipa Spp.)', 'https://www.museodelprado.es/coleccion/obras-de-arte?cidoc:p14_carrie

In [91]:
columns = ['Titulo', 'Tecnica_Fecha', 'Autor', 'Flora', 'Href']
flora_df_tulipan = pd.DataFrame(flora_data_tulipan, columns=columns)
flora_df_vid_parra = pd.DataFrame(flora_data_vid_parra, columns=columns)

In [92]:
flora_df_tulipan

,Titulo,Tecnica_Fecha,Autor,Flora,Href
0,La Tierra,Óleo sobre lienzo. Siglo XVI,"VOS, MARTIN DE",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
1,Guirnalda con la Adoración de los Reyes Magos,Óleo sobre lámina de cobre. Siglos XVI - XVII,"BRUEGHEL EL JOVEN, PIETER",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
2,Vase of Flowers,Óleo sobre tabla. 1600 - 1625,"BRUEGHEL EL VIEJO, JAN",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
3,Florero,Óleo sobre tabla. Primer cuarto del siglo XVII,"BRUEGHEL EL VIEJO, JAN",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
4,Florero,Óleo sobre lienzo. Primer cuarto del siglo XVII,"BRUEGHEL EL VIEJO, JAN",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
5,Florero,Óleo sobre lienzo. Primer cuarto del siglo XVII,"BRUEGHEL EL VIEJO, JAN",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
6,"Guirnalda de flores con la Virgen, el Niño y S...",Óleo sobre lienzo. Primera mitad del siglo XVII,"QUELLINUS, ERASMUS ; SEGHERS, DANIEL",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
7,Florero,Óleo sobre lienzo. Siglo XVII,"ARELLANO, JUAN DE",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
8,Florero,Óleo sobre lienzo. Siglo XVII,"ARELLANO, JUAN DE",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...
9,Florero,Óleo sobre lienzo. Siglo XVII,"PÉREZ, BARTOLOMÉ (ATRIBUIDO A)",Tulipán (Tulipa Spp.),https://www.museodelprado.es/coleccion/obras-d...


In [4]:


def obtener_enlaces(xpath, url):
    
    try:
        # Abrir la página web
        driver.get(url)
        
        # Esperar a que los elementos se carguen (puedes ajustar el tiempo según tus necesidades)
        driver.implicitly_wait(10)
        
        # Encontrar todos los elementos con el XPath dado
        elementos = driver.find_elements(By.XPATH, xpath)
        
        # Obtener los enlaces (href) de cada elemento
        enlaces = [elemento.get_attribute("href") for elemento in elementos]
        
        return enlaces
    
    finally:
        # Cerrar el navegador al finalizar
        driver.quit()

# Ejemplo de uso:
xpath_ejemplo = '//*[@id="modal-autor"]/div[2]/ul[1]/li[1]/a'
url_ejemplo = 'https://www.museodelprado.es/coleccion/obras-de-arte'

try:
    enlaces_resultantes = obtener_enlaces(xpath_ejemplo, url_ejemplo)
    print(enlaces_resultantes)
except Exception as e:
    print(f"Error: {e}")


[]


In [4]:
def scrapeo_flora_data(driver, url):
    driver.get(url)

    # Obtener el tipo de flora de la etiqueta en la parte superior de la página
    tipo_flora = driver.find_element(By.XPATH, '//*[@id="panListadoFiltros"]/ul/li').text.strip()

    figures = driver.find_elements(By.XPATH, '//*[@id="panResultados"]/div/figure')

    resultados = []

    for figure in figures:
        titulo = figure.find_element(By.XPATH, './/figcaption[1]/div/p[1]').text.strip()
        fecha = figure.find_element(By.XPATH, './/figcaption[1]/div/p[2]').text.strip()
        autor = figure.find_element(By.XPATH, './/figcaption[1]/div/p[3]').text.strip()

        try:
            href = figure.find_element(By.XPATH, './/figcaption[1]/div/p[3]/a').get_attribute('href')
        except NoSuchElementException:
            # Manejo de excepción si no se encuentra el elemento href
            href = None

        resultados.append([titulo, fecha, autor, tipo_flora, href])

    return resultados

# Ejemplo de uso
urls = [
    'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_298',
    'https://www.museodelprado.es/coleccion/obras-de-arte?searchObras=vid%20/%20parra%20(vitis%20vinifera)',
    # Agrega más URLs según sea necesario
]

# Lista para almacenar los resultados de todas las flores
resultados_globales = []

# Ejecutar la función para cada URL de flor
for url_flor in urls:
    resultados_flor = scrapeo_flora_data(driver, url_flor)
    resultados_globales.extend(resultados_flor)

# Convertir la lista de resultados globales en un DataFrame
columns = ['Titulo', 'Tecnica_Fecha', 'Autor', 'Flora', 'Href']
flora_df_global = pd.DataFrame(resultados_globales, columns=columns)

# Imprimir el DataFrame
flora_df_global


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="panListadoFiltros"]/ul/li"}
  (Session info: chrome=120.0.6099.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100bd84dc chromedriver + 4162780
1   chromedriver                        0x0000000100bd0664 chromedriver + 4130404
2   chromedriver                        0x0000000100827bc0 chromedriver + 293824
3   chromedriver                        0x000000010086e040 chromedriver + 581696
4   chromedriver                        0x00000001008adab0 chromedriver + 842416
5   chromedriver                        0x00000001008616bc chromedriver + 530108
6   chromedriver                        0x0000000100862930 chromedriver + 534832
7   chromedriver                        0x0000000100b9de08 chromedriver + 3923464
8   chro

In [69]:
def obtener_hrefs(driver, xpath):
    # Espera implícita para asegurarse de que la página esté completamente cargada
    driver.implicitly_wait(10)
    
    # Espera explícita para asegurarse de que los elementos estén presentes antes de buscarlos
    wait = WebDriverWait(driver, 10)
    elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
    
    hrefs = []
    
    for elemento in elementos:
        try:
            # Obtén el enlace (href) dentro del elemento
            href = elemento.find_element(By.XPATH, './/a').get_attribute('href')
            hrefs.append(href)
        except NoSuchElementException:
            # Manejo de excepción si no se encuentra el elemento href
            print("No se encontró el elemento href en:", elemento.text)
    
    return hrefs

# Ejemplo de uso
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'
xpath_elementos = '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li'

# Obtener hrefs
hrefs = obtener_hrefs(driver, xpath_elementos)

# Imprimir los hrefs obtenidos
print(hrefs)


['https://www.museodelprado.es/coleccion/obras-de-arte', 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_109', 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_111', 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_116', 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://mus

In [70]:
for href in hrefs:
    print(href)

https://www.museodelprado.es/coleccion/obras-de-arte
https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_109
https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_111
https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_116
https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/